In [1]:
import requests, re, time

zhihuUrl = "http://www.zhihu.com"
username = "zord-vczh"
email = ""
password = ""
cookie = ""

login_data = {
    'email': email,
    'password': password
}

s = requests.session()
s.post(zhihuUrl + '/login/email', login_data)

s.headers.update({'Connection':'keep-alive'})
s.headers.update({'Cookie':cookie})
s.headers.update({'DNT':1})
s.headers.update({'Host':"www.zhihu.com"})
s.headers.update({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/43.0.2357.130 Chrome/43.0.2357.130 Safari/537.36'})
s.headers.update({'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'})

In [14]:
r = s.get(zhihuUrl + '/people/' + username + '/answers')

f = open(username + '_answerpage.html', 'w')
f.write(r.content)
f.close()

pagePattern = re.compile(r'page=\d+">\d+</a>')
foundList = pagePattern.findall(r.content)

maxPage = 0
if len(foundList) > 0:
	maxPagePattern = re.compile(r'\d+')
	for item in foundList:
		tmp = maxPagePattern.findall(item)
		if len(tmp)	> 0 and int(tmp[0]) > maxPage:
			maxPage = int(tmp[0])

print maxPage

427


In [19]:
answerPageUrl = zhihuUrl + "/people/" + username + "/answers?page="
answerLinkPattern = re.compile(r'<a class="question_link" href="(.*)">')
approvePattern = re.compile(r'<div class="zm-item-vote-info " data-votecount="(\d+)">')

answerLinks = [] 

f = open(username + '_answerLinks.csv', 'w')
count = 0

for i in range(0, maxPage):
	preUrl = answerPageUrl + str(i+1)
	preR = s.get(preUrl)
	if preR.status_code == 200:
		preContent = preR.content
		tmp = answerLinkPattern.findall(preContent)
		approveNum = approvePattern.findall(preContent)
		while len(tmp) - len(approveNum) < 0:
			tmp.append("error")
		while len(tmp) - len(approveNum) > 0:
			approveNum.append("0")
		for j in range(0, len(tmp)):
			f.write(zhihuUrl + tmp[j] + ',' + approveNum[j] + '\n')
			answerLinks.append(zhihuUrl + tmp[j])
			count = count + 1;
	else:
		count = count + len(tmp)
	if count % 600 == 0:
		print count
f.close()

600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600
7200
7800
8400


In [2]:
f = open(username + '_answerLinks.csv', 'r')
links = f.readlines()
f.close()

In [3]:
questionPattern = re.compile(r'<title>(.*?)</title>')
answerPattern = re.compile(r'zm-editable-content clearfix">((.|\n)*?)</div>')
possibleTimePattern = re.compile(r'<a class="answer-date-link (.*) href=(.*?)</a>')
timePattern = re.compile(r' ((\d|-|:)*)')
shortLinkPattern = re.compile(r'"(/question/.*?)"')

In [5]:
f = open(username + '_answerInfos.csv', 'w')
fD = open(username + '_answerDetails.csv', 'w')

for i in range(0, len(links)):
    preLink = links[i].split(",")[0]
    preR = s.get(preLink)
    question = "error"
    answer = "error"
    answerTime = "error"
    shortLink = "error"
    approveNum = links[i].split(",")[1][:-1]

    if preR.status_code == 200:
        preContent = preR.content
        question = questionPattern.findall(preContent)
        question = (question[0][:-26] if len(question) != 0 else "error")
        answer = answerPattern.findall(preContent)
        answer = (answer[0][0][:-3] if len(answer) != 0 else "error")
        possibleTimeStr = possibleTimePattern.findall(preContent)
        for itemsen in possibleTimeStr:
            for item in itemsen:
                for term in shortLinkPattern.findall(item):
                    if term in preLink:
                        answerTime = timePattern.findall(item)
                        if '-' in answerTime[0][0]:
                            answerTime = answerTime[0][0]
                        else:
                            answerTime = answerTime[1][0]
                        shortLink = term
                        if ":" in answerTime:
                            answerTime = time.strftime('%Y-%m-%d ') + answerTime
                        #print shortLink, answerTime, timePattern.findall(item)

    f.write(str(i) + "," + shortLink + "," + approveNum + "," + answerTime + "\n")
    fD.write(str(i) + ",|" + shortLink + ",|" + answer + "]>]>]>")
    if i % 600 == 0:
        print i

f.close()
fD.close()

0
600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600
7200
7800
8400
